In [2]:
import numpy as np

np.seterr(all="ignore")


In [18]:
problem = np.load('../data/problem_1.npz')
x = problem['x']
y = problem['y']

num_variables= x.shape[0]

print(y)

[ 0.76750749 -0.8259928  -0.09060874  0.18618096  0.07540174  0.62988407
 -0.57561624  0.7637362  -0.57202814  0.41926627 -0.16731469 -0.52369952
  0.69781335  0.5775741   0.63213487 -0.65729413 -0.8215542   0.64392269
 -0.49960976 -0.67738703 -0.23591522 -0.31453371 -0.15187967 -0.19741723
 -0.5923385  -0.02303459  0.62871104  0.76473999  0.14558534  0.80502106
  0.82257038 -0.82442055  0.5349135   0.69317175  0.38409531 -0.44029759
  0.82152816 -0.43751907  0.3438024  -0.28376748 -0.07830488  0.63247381
  0.36927498  0.13495229 -0.0670447  -0.26530326 -0.39952276  0.56039517
 -0.43416159  0.33489672  0.83236445  0.15192441  0.09955559  0.75952645
  0.46420294  0.26813435 -0.69359681 -0.57432375  0.35022553 -0.03131206
  0.10914955 -0.16413869  0.81848374  0.58177478 -0.73631196  0.11875495
  0.2427744   0.77805089 -0.33338925 -0.43547337 -0.15992821  0.76247319
  0.19549622  0.28947934 -0.60891747  0.77630662  0.56493027 -0.02990157
  0.64059138  0.47981529  0.34347079  0.74335292  0